# Coursera Capstone Course

## About this repository
This repository represents the final assignment in the coursera certification course "IBM Data Science Professional Certificate" (See: https://www.coursera.org/professional-certificates/ibm-data-science).

In [99]:
import numpy as np
import pandas as pd

import requests
import codecs

import folium

print("Hello Capstone Project Course")

Hello Capstone Project Course


In [100]:
# Imports

In [101]:
data = pd.read_json(".private_data.json")["FOURSQUARE"]

# Authentification data
CLIENT_ID = data["ID"]
CLIENT_SECRET = data["SECRET"]

# Parameters
LIMIT = 50
RADIUS = 500 # 500 meters within range of location

# Code url fragments
url_base = "https://api.foursquare.com/v2/venues/search"
url_client = f"&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}"


In [102]:
SOURCE_LINK_VVS = "https://www.openvvs.de/dataset/1f7ec4c1-b590-4751-a25b-57ef988c14b6/resource/d87d1f01-5c14-4d08-8452-e405a6472ab4/download/vvs_haltestellen.csv"

df_vvs = pd.read_csv(SOURCE_LINK_VVS, delimiter=";", encoding="iso-8859-1")

In [103]:
df = df_vvs

# Extract all possible ways of commotion
vm = list(df["Verkehrsmittel"].unique())
vm_unique = set()

for y in vm:
    for x in y.split(";"):
        vm_unique.add(x) 

print(vm_unique)


{'Linientaxi', 'Stadtbahn', 'Nachtbus', 'SEV-Bus', 'Zahnradbahn', 'Ruftaxi', 'R-Bahn', 'Bus', 'Expressbus', 'Bürgerbus', 'Seilbahn', 'S-Bahn'}


In [104]:
        
df = df[df["Verkehrsmittel"].str.match(".*S-Bahn.*")].reset_index()

print(df.shape)
df.head()

(84, 17)


,index,Nummer,Name,Name mit Ort,Globale ID,GKZ,Ort,Teilort,Landkreis,Tarifzonen,Verkehrsmittel,Linien (EFA),Linien (DIVA),Anzahl Linien,Betriebszweige,X-Koordinate,Y-Koordinate
0,13,34,Nürnberger Straße,Nürnberger Straße,de:08111:34,8111000,Stuttgart,Stuttgart,S,20,S-Bahn;R-Bahn;Nachtbus;Stadtbahn,"S2,S3,R2,R3,U1,N6","10002_,10003_,11002_,11003_,20001_,33006_",6,"10,11,20,33","9,234576905","48,80656049"
1,84,175,Leinfelden,Leinfelden,de:08116:175,8116078,Leinfeld.-Echterd.,Leinfelden,ES,38,Bus;S-Bahn;Stadtbahn,"S2,S3,S6,U5,35,38,E,82,86,826,826A","10002_,10003_,10006_,20005_,30035_,30038_,3003...",11,"10,20,30,31","9,142859698","48,69641917"
2,241,1003,Höfingen,Höfingen,de:08115:1003,8115028,Leonberg,Höfingen,BB,"36,46",S-Bahn,"S6,S60","10006_,10060_",2,10,"9,026180834","48,81503423"
3,243,1300,Sommerrain,Sommerrain,de:08111:1300,8111000,Stuttgart,Stuttgart,S,20,Bus;R-Bahn;S-Bahn,"S2,S3,R2,R3,54,58","10002_,10003_,11002_,11003_,30054_,30058_",6,"10,11,30","9,24834265","48,81524515"
4,244,1301,Malmsheim,Malmsheim,de:08115:1301,8115041,Renningen,Malmsheim,BB,56,S-Bahn,S6,10006_,1,10,"8,905105557","48,77183551"


In [105]:
df["Name"].values

array(['Nürnberger Straße', 'Leinfelden', 'Höfingen', 'Sommerrain',
       'Malmsheim', 'Rutesheim', 'Weil der Stadt', 'Bietigheim',
       'Kornwestheim', 'Neuwirtsh. (Porschep.)', 'Benningen (N)',
       'Freiberg (N)', 'Nellmersbach', 'Neustadt-Hohenacker',
       'Schwaikheim', 'Stetten-Beinstein', 'Geradstetten', 'Grunbach',
       'Weiler (R)', 'Winterbach', 'Altbach', 'Mettingen',
       'Oberesslingen', 'Zell', 'Filderstadt', 'Flughafen/Messe',
       'Oberaichen', 'Weilimdorf Bf', 'Süd', 'Magstadt', 'Nord',
       'Maichingen', 'Sindelfingen', 'Goldberg', 'Erdmannhausen',
       'Beutelsbach', 'Kirchheim (T)', 'Wernau (N)', 'Wendlingen (N)',
       'Ötlingen', 'Herrenberg', 'Ehningen', 'Gärtringen', 'Nufringen',
       'Rohr', 'Vaihingen', 'Universität', 'Österfeld', 'Schwabstraße',
       'Stadtmitte', 'Neckarpark', 'Untertürkheim', 'Obertürkheim',
       'Hauptbahnhof (oben)', 'Hauptbahnhof (tief)', 'Feuerbach',
       'Feuersee', 'Nordbahnhof', 'Bad Cannstatt', 'Zuffenhause

In [138]:
# Create a copy of the original dataframe
df_plot = df.loc[:, ["Name", "Y-Koordinate", "X-Koordinate"]]
df_plot.columns = ["Name", "Latitude", "Longitude"]

# Rectify some german conventions
formater = lambda x: x.replace(",", ".")
df_plot["Latitude"] = df_plot["Latitude"].apply(formater).astype(np.float64)
df_plot["Longitude"] = df_plot["Longitude"].apply(formater).astype(np.float64)

# Get mean lat and long as starting point
latitude = df_plot["Latitude"].mean()
longitude = df_plot["Longitude"].mean()

# Create the map and the labels
map = folium.Map(location = (latitude, longitude), zoom_start=10)
for _, (name, lat, long) in df_plot.iterrows():
    label = folium.Popup(name)
    
    folium.CircleMarker(
        location=(lat,long),
        popup=label,
        color="red",
        fill=True,
        radius=10,
        fill_opacity=0.2
    ).add_to(map)

map